<h1>Running Customized Moebius Simulations</h1>

This example is designed to show how it can be possible to submit Moebius simulations with customized input files on the OnScale cloud through a combination of operations carried out via the OnScale solve UI (https://solve.portal.onscale.com) and the OnScale python client.


<h2>Setting up a Model</h2>

The first steps of this example are carried out using the OnScale Solve UI. Having logged in using a username and password, the user should setup a Moebius project with the desired model setup. 

This is necessary to allow us to initialize the project within the OnScale system and be able to visualize the results within the OnScale Post Processor.

<h2>Estimate the Model</h2>

Having completed the model setup, we need to carry out the Mesh generation and Estimate step of the simulation workflow by pressing the 'Mesh & Estimate' button in the OnScale Solve UI.

Carrying out this step will create a Job object which can then be accessed using the OnScale python client. Once the job is instantiated the Mesh that will be used within the simulation will be generated and stored within a folder belonging to this Job in OnScale's cloud storage.
Once the mesh has been generated, the input files which will be ran on the Moebius solver will be generated and stored alongside the mesh within the cloud storage.

Once the estimate is completed, we are then ready to move to the OnScale python client.


<h2>Configuration</h2>

Before proceeding with this example, please ensure you have followed all steps laid out in the `Dev Setup` and `Configuration` sections of `packages/cloud_client/README.md`.



<h2>Imports</h2>

Import the onscale_client module, giving it an appropriate alias which can be used going forwards. We also import the os module to allow some file path operations to be completed.

In [ ]:
import onscale_client as cloud
import os

<h2>Setup Global Variables</h2>

Initialise a few global variables which will be used throughout the workflow. These global variables will be used to identify:
<html><ul>
  <li>The PORTAL on which the Moebius job should be ran</li>
  <li>The JOB_ID (if necessary) of the Moebius job which has been created  and estimated via the OnScale Solve UI</li>
  <li>And the (DL_DIR) directory to Download the input file which will be updated before submitting the job</li>
</ul></html>

In [ ]:
# Set the portal you have setup the moebius job and estimated using
# the OnScale Solve UI
PORTAL = 'Test'
# specify the JobId which corresponds to the model estimated on solve
JOB_ID = ''
# specify the download directory. os.getcwd() specifies the current
# working directory
DL_DIR = os.getcwd()

It is possible to view the available profiles which have been configured for use on to alow us to select the correct one before logging in. We do this by calling the `get_available_profiles` function and printing out the results.

In [ ]:
#
profiles = cloud.get_available_profiles(portal_target=PORTAL)
print(profiles)


Once we have identified the profile to use for logging in, we should instantiate an instance of the `onscale_client.Client` object. This `Client` object will connect to the OnScale cloud platform to allow access to jobs which have already been ran and allow a user to create and submit new jobs.

In [ ]:
# Create a Client instance which will use the desired profile for
# the given target. You must ensure the profile selected is the
# same as the account which set up the model.
client = cloud.Client(alias=profiles[0], portal_target=PORTAL)


Once we have a `Client` instance which is logged in to the cloud, we want to get the job object for the desired job we want to edit.  If this job is the last job which the user has created, the `get_last_job` function can be called, otherwise, the `get_job` function can be called using the specific `JOB_ID` for the desired job.

If the JOB_ID is unknown, then it is possible to see a recent history of jobs using the `get_job_history` function. See the `onscale_client` documentation for details of this.

In [ ]:
# request the last job which has been created
job = client.get_last_job()

# request a job using a specific JOB_ID
#job = client.get_job(job_id=JOB_ID)

With a job object instantiated, it is possible to download the input file which will be ran for this job using `Job.download_file`.  The input file is always named using the job_id so we can use that information to specify the file which will be downloaded. 

It is also possible to get a list of the files available for a job using the `Job.file_list` and `Job.blob_list` functions.

In [ ]:
# download the input file for the created job
job.download_file(file_name=f'{job.job_id}.py', download_dir=DL_DIR)


At this point, it should be possible to edit the Moebius input file to include any functionlality that may be required. Once the input file is complete, it should then be uploaded using the `Job.upload_file` function. This will overwrite the file which already exists for this job on the cloud.

In [ ]:

# upload the edited file to the job
job.upload_file(filename=os.path.join(DL_DIR, f'{job.job_id}.py'))


If the updates to the input file now require additional files to be included alongside the job, these can be uploaded along side the input file using the same `Job.upload_file` function.

In [ ]:
# upload any accompanying files which may now be required to complete
# the simulation
job.upload_file(filename='path_to_accompanying_file')

Once all files have been uploaded, it should now be possible to return to the OnScale Solve UI, Run the simulation and view the results.